Load the dataset

In [ ]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [ ]:
with open('input.text', 'r', encoding='utf-8') as f:
    text = f.read()

In [ ]:
print("length of dataset in characters: ", len(text))

In [ ]:
print(text[:1000])

In [ ]:
# here are all the unique characers that occur in this text
chars = sorted(list(set(text)))
# set constructor on text, and converted to a sorted list
vocab_size = len(chars)
print(''.join(chars))
# all possible chars model can utilize
print(vocab_size)

In [ ]:
# Character tokenizer
# create a mapping from characters to integers
stoi = { ch:i for i, ch in enumerate(chars)}
itos = { i:ch for i, ch in enumerate(chars)}
encode = lamda s: [stoi[c] for c in s] # encoder: take a string, output a list of string
decode = lamda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

In [ ]:
import torch 
data = torch.tensor(encode(text), dtype = torch.long))
print(data.shape, data.dtype)
print(data[:1000])

In [ ]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest eval
train_data = data[:n]
val_data = data[n:]

In [ ]:
block_size = 8
train_data[:block_size+1]

In [ ]:
x = train_data[:block_size]
y = train_data[1:block_size=1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f'when input is {context} the target: {target}')

In [ ]:
# By initializing the random no generator we get same output as karpathy's
# random locations from data where we will take blocks from
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel>
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
  # generate a small batch of data of inputs x and targets y
  data = train_data if split == 'train' else val_data
  # random block sizes of size 4 generateed ix
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack(data[i: i+block_size] for i in ix)
  y = torch.stack(data[i+1: i+block_size+1] for i in ix)
  return x, y

  xb, yb = get_batch('train')
  print("inputs:")
  print(xb.shape)
  print(xb)
  print("targets:")
  print(yb.shape)
  print(yb)

  print('-------')

  for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
      context = xb[b, :t+1]
      target = yb[b, t]
      print(f"when input is {context.tolist()} the target: {target}")

bigram language model

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as f
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
  super().__init__()
  # constructor vocabsize*vocabsize
  # each token directly reads off teh logits for the next token from a lookup table
  self.token_embedding_table = nn.Embedding(vocab_size, vaocab_size)

# idx = index
def forward(self, idx, targets = None):

  # idx and targets are both (B, T) tensor of integers 
  logits = self.token_embedding_table(idx) # (Batch, Time, Channels)
  if targets is None:
    loss = None
  
  else:
    B, T, C = logits.shape
    # 2D array stretching out to a 2D array
    logits = logits.view(B*T, C)
    targets = targets.view(B*T)
    # targets = targets.view(-1) pytorch does it automaticallsy
    loss = F.cross_entropy(logits, targets)
  
  return logits, loss

def generate(self, idx, max_new_tokens):
  # get the predictions only logit part works
  logits, loss = self(idx)
  # focus only on the last time step
  logits = logits[:, -1, :] # becomes [B, C]
  # apply softmax to get probabilities
  probs = F.softmax(logits, dim = 1) # [B, C]
  # sample from the distribution
  idx_next = torch.multimonial(probs, num_samples = 1) # (B, 1)
  # append sampled index to the running sequence
  idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
return idx

m = BigramLanguageModel(vocab_size)
logits, out = m[xb, yb]
print(out.shape)
print(logits.shape)
print(loss)
idx = torch.zeros((1, 1), dtype = torch.long)
print(decode([m.generte](, max_new_tokens=100)[0].tolist()))
# print(logits.shape) is not possible coz crossentropy requires B, c, t

In [ ]:
# create a PyTorch optimizer
# Stochastic Gradient Descent - SGD
# small networks you can getaway with much higher lr
optimizer = torch.optim.Adam(m.parameters(), lr = 1e-3)


In [ ]:
# opitimization object
batch_size = 32
for steps in range(1000):

  # sample a batch of data
  xb, yb = get_batch['train']

  # evaluate the loss 
  logits, loss = m[xb, yb] # evaluate the loss
  optimizer.zero_grad(set_to_none=True) # zeroing all the gradients
  loss.backward() # getting gradients for all parameter
  optimizer.step() # update the parameters
print(loss.item())